In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [3]:
# --- CONFIGURATION (ADJUST THESE) ---
IMAGE_FOLDER = '/Users/gagan/Machine learning/Eye project/Fundus_CIMT_2903 Dataset'          # Folder containing the images
DATA_FILE = 'data_info.csv' # Your metadata file (Check if it's .csv or .xlsx)
IMG_SIZE = 128                   # Resize images to 128x128
BATCH_SIZE = 32

In [4]:
def load_and_process_data():
    print(f"Reading data from {DATA_FILE}...")
    
    # Load the CSV
    df = pd.read_csv(DATA_FILE)
    
    # Print first few rows to confirm it works
    print(f"Found {len(df)} patients in the CSV.")
    
    images = []
    targets = [] 

    print("Loading images... (This might take a minute)")
    
    # Loop through every patient in the CSV
    count = 0
    for index, row in df.iterrows():
        
        # 1. Get the Target (We use 'True_age' for Age Prediction)
        # CHANGE THIS TO row['thickness'] IF YOU WANT TO PREDICT HEART RISK
        patient_target = row['True_age'] 
        
        # 2. Get Filenames for BOTH eyes
        files_to_check = [ row['right_eye'], row['left_eye'] ]
        
        # 3. Load both images
        for filename in files_to_check:
            # Construct full path
            img_path = os.path.join(IMAGE_FOLDER, filename)
            
            # Only load if the file actually exists
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                
                # --- Preprocessing ---
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                img = img[:, :, 1] # Keep Green Channel only (Best for vessels)
                img = img / 255.0  # Normalize (0-1)
                
                # Reshape to (128, 128, 1) for the AI
                img = np.expand_dims(img, axis=-1)
                
                images.append(img)
                targets.append(patient_target)
                count += 1
            else:
                pass

    print(f"Successfully loaded {count} images (Left + Right combined).")
    return np.array(images), np.array(targets)

In [5]:
def build_model():
    """
    Standard CNN for Regression
    """
    model = Sequential([
        # Layer 1
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Layer 2
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Layer 3
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        Flatten(),
        
        # Dense Layers (Thinking layers)
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        
        # Output: 1 single number (The Age)
        Dense(1, activation='linear') 
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
# --- MAIN EXECUTION ---
if __name__ == "__main__":
    
    # 1. Load Data
    X, y = load_and_process_data()
    
    if len(X) > 0:
        # 2. Split Data (80% Train, 20% Test)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # 3. Build & Train
        model = build_model()
        
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        print(f"Starting training on {len(X_train)} images...")
        model.fit(
            X_train, y_train,
            epochs=30,
            batch_size=BATCH_SIZE,
            validation_data=(X_test, y_test),
            callbacks=[early_stop]
        )
        
        # 4. Results
        print("\n--- Final Test Results ---")
        loss, mae = model.evaluate(X_test, y_test)
        print(f"Average Error: {mae:.2f} years")
        
        # 5. Save
        model.save('age_model.h5')
        print("Model saved as age_model.h5")
        
    else:
        print("Error: No images found. Please check your 'images' folder path.")


Reading data from data_info.csv...
Found 2903 patients in the CSV.
Loading images... (This might take a minute)
Successfully loaded 5806 images (Left + Right combined).
Starting training on 4644 images...
Epoch 1/30


/Users/gagan/Machine learning/Eye project/age-and-heart-attack-prediction/venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
